# UMTA Experiment 1: Architecture Search (Kaggle Notebook - Parallel Execution)

This notebook runs **Experiment 1** with parallel execution support.

## Parallel Execution Strategy
- **10 separate Kaggle notebooks** (one per pair)
- Each notebook runs **1 pair** across **4 architectures**
- Set `PAIR_INDEX` in Cell 3 to select which pair to run

**Repo:** https://github.com/hanjidani/UMTA_Final_Experiments


In [ ]:
# Cell 1: Install Dependencies
# Install required libraries for CLIP and training

!pip install -q ftfy regex tqdm
!pip install -q git+https://github.com/openai/CLIP.git


In [ ]:
# Cell 2: Clone Repository

import os

# Remove existing repo to ensure we get the latest code
if os.path.exists("UMTA_Final_Experiments"):
    !rm -rf UMTA_Final_Experiments

# Clone the repository
!git clone https://github.com/hanjidani/UMTA_Final_Experiments.git

# Move into the directory
%cd UMTA_Final_Experiments

print("✅ Repository cloned and ready")


In [ ]:
# Cell 3: Unify Kaggle Dataset Structure
# This cell handles Kaggle's split dataset folders (train.X1, train.X2, etc.)
# and merges them into a single unified structure using symbolic links

import os
import yaml

# Define paths
input_root = "/kaggle/input/imagenet100"
unified_root = "/tmp/imagenet100"  # Unified dataset location

print(f"🛠️ Fixing Kaggle dataset structure from {input_root}...")

# Create destination folders
os.makedirs(f"{unified_root}/train", exist_ok=True)
os.makedirs(f"{unified_root}/val", exist_ok=True)

# Merge 'train.X*' folders into a single 'train' folder using symlinks
# This makes Python think it's one big folder without actually copying files
!cp -rs {input_root}/train.X*/* {unified_root}/train/ 2>/dev/null || echo "No train.X* folders found, checking for single train folder..."
!cp -rs {input_root}/val.X*/* {unified_root}/val/ 2>/dev/null || echo "No val.X* folders found, checking for single val folder..."

# Fallback: If no split folders, try direct copy/symlink
if not os.listdir(f"{unified_root}/train"):
    !cp -rs {input_root}/train/* {unified_root}/train/ 2>/dev/null || true
if not os.listdir(f"{unified_root}/val"):
    !cp -rs {input_root}/val/* {unified_root}/val/ 2>/dev/null || true

print("✅ Dataset unified successfully!")

# Update config file to point to unified location
config_path = 'exp1_architecture/config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Point config to the new unified location
if 'data' not in config:
    config['data'] = {}
config['data']['path'] = unified_root

with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"✅ Config updated to read from: {unified_root}")


In [ ]:
# Cell 4: Select Pair Index (USER INPUT)
# ==========================================
#        KAGGLE PARALLEL CONFIGURATION
# ==========================================
# Set this index from 0 to 9 to run a specific pair.
# Open 10 separate Kaggle tabs and assign a unique index to each.
#
# Pairs:
# 0: Fish -> Shark       (Easy)
# 1: Finch -> Bunting    (Easy)
# 2: Plant -> Frog       (Med-Easy)
# 3: Dog -> Cat          (Med-Easy)
# 4: Truck -> Car        (Medium)
# 5: Snake -> Lizard     (Medium)
# 6: Bird -> Object      (Med-Hard)
# 7: Bird -> Artifact    (Med-Hard)
# 8: Fish -> Paper       (Hard)
# 9: Fish -> Traffic Light (Hard)
# ==========================================

PAIR_INDEX = 0  # <--- CHANGE THIS VALUE (0-9)


In [ ]:
# Cell 5: Run Experiment
# Execute the training script with the selected pair index
# The script handles architecture creation, training, and saving results.

print(f"Starting Experiment 1 for Pair Index: {PAIR_INDEX}")
!python exp1_architecture/run.py --pair_index {PAIR_INDEX}


In [ ]:
# Cell 6: Download Results
# Create a download link for the results CSV

from IPython.display import FileLink, display
import os
from pathlib import Path

# Find the latest results directory
results_base = Path("exp1_architecture/results")
if results_base.exists():
    # Get the most recent timestamped directory
    result_dirs = sorted([d for d in results_base.iterdir() if d.is_dir()], reverse=True)
    if result_dirs:
        latest_dir = result_dirs[0]
        result_file = latest_dir / "results.csv"
        
        if result_file.exists():
            print(f"✅ Experiment Complete!")
            print(f"Results saved in: {latest_dir}")
            print(f"\nClick below to download results for Pair {PAIR_INDEX}:")
            display(FileLink(str(result_file)))
        else:
            print(f"⚠️ Results directory found but results.csv not found in {latest_dir}")
    else:
        print("⚠️ No results directories found")
else:
    print("⚠️ Results directory not found. Check the training logs above for errors.")
